In [1]:
# dependencies
!pip install psutil numpy pillow tensorflow statistics

import time
import psutil
import numpy as np
from PIL import Image
import tensorflow as tf
import statistics

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 1.6 MB/s eta 0:00:00-:--:--
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7491 sha256=8a9bb30ef9840eb0edce7d14bc21b34a79be08547b28193255b94754588ace3c
  Stored in directory: /Users/belix/Library/Caches/pip/wheels/10/9c/1a/0c68a89e4533a18e9f9902018b94ffc2481139818b67cbb05a
Successfully built statistics

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


2025-05-29 03:25:29.480033: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## MobileNetV2

In [2]:
# Loading the TFLite model
interpreter = tf.lite.Interpreter(model_path="models/mobilenetv2.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Model loaded successfully.")

Model loaded successfully.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Load and Preprocess Sample Image

In [3]:
image = Image.open("test_images/test_dog.jpeg").resize((224, 224))
input_data = np.expand_dims(np.array(image) / 255.0, axis=0).astype(np.float32)
print("Image loaded and processed.")

Image loaded and processed.


## Inference & Performance

In [4]:
input_scale, input_zero_point = input_details[0]['quantization']
input_data_uint8 = (input_data / input_scale + input_zero_point).astype(np.uint8)

inference_times = []

# Run 100 times for benchmarking
for _ in range(100):
    start_time = time.time()

    interpreter.set_tensor(input_details[0]['index'], input_data_uint8)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])
    elapsed = (time.time() - start_time) * 1000  # milliseconds
    inference_times.append(elapsed)

# Calculate stats
avg_time = sum(inference_times) / len(inference_times)
std_time = statistics.stdev(inference_times)

# RAM usage
ram_usage = psutil.Process().memory_info().rss / 1024 / 1024  # MB

# Output results
print(f"Average Inference Time: {avg_time:.2f} ms")
print(f"Standard Deviation: {std_time:.2f} ms")
print(f"RAM Usage: {ram_usage:.2f} MB")

Average Inference Time: 17.71 ms
Standard Deviation: 5.06 ms
RAM Usage: 403.06 MB


## Log Result

In [5]:
import csv

with open("benchmark_log.csv", mode="a") as file:
    writer = csv.writer(file)
    writer.writerow(["MobileNetV2_Quantized", f"{inference_time:.2f}", f"{ram_usage:.2f}"])

NameError: name 'inference_time' is not defined